In [66]:
import pandas as pd
import string
import re
import spacy
from string import punctuation
Spacy_Eng = spacy.load('en_core_web_sm')
Spacy_Eng_STOP_WORDS = Spacy_Eng.Defaults.stop_words
import nltk
from nltk.stem.porter import PorterStemmer
p_stemmer = PorterStemmer()
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [2]:
df = pd.read_csv("gathered-products-detail.csv", encoding='latin1') # df

In [7]:
df.head(2) # df

,docID,availabilityStatus,averageRating,brand,model,shortDescription,id,manufacturerName,name,orderMinLimit,...,reviews.roundedAverageOverallRating,reviews.topNegativeReview.reviewSubmissionTime,reviews.topNegativeReview.userNickname,reviews.topNegativeReview.reviewText,reviews.topNegativeReview.reviewTitle,reviews.topPositiveReview.reviewSubmissionTime,reviews.topPositiveReview.userNickname,reviews.topPositiveReview.reviewText,reviews.topPositiveReview.reviewTitle,reviews.totalReviewCount
0,0,IN_STOCK,4.1,Eliminator,HG-41352,"[[Eliminator Ant, Roach & Spider Killer4 is a ...",5D3NBXRMIZK4,Eliminator,"Eliminator Ant, Roach & Spider Killer4, 20 oz,...",1,...,"[[4.1],[4.5],[]]","[[1/28/2022],[],[]]","[[Tom],[],[]]",[[Covered one up in phone. The roach crawled o...,"[[Ineffective against common brown cockroach],...","[[4/5/2020],[4/5/2019],[]]","[[amity],[shaziabana],[]]",[[With everything going on in the world you wo...,"[[Social distancing should include spiders.],[...","[[119],[200],[]]"
1,1,IN_STOCK,NaN,Spider-Man,78789,[[Celebrate like a superhero using our Spiderm...,158SLECVHWMQ,Unique Industries,"Spiderman Party Blowers, 8ct",1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[0],[],[]]"


In [13]:
product_names = [] # list[]
docs = [] # list[]
for index, row in df.iterrows(): # int, series
    stringDoc = '' # str
    for key in df.columns: # str
        if key == 'name': # exact match
            product_names.append(row[key]) # list[str]
        stringDoc += ' ' + re.sub("[,\[\]]", " ", str(row[key])) # list[str]
    docs.append(stringDoc) # list[str]


In [38]:
docs

[" 0 IN_STOCK 4.1 Eliminator HG-41352   Eliminator Ant  Roach & Spider Killer4 is a water-based  non-staining formula that kills on contact. This product give you up to eight week control and leaves no lingering odor. Use this product to kill cockroaches  ants  carpenter ants  crickets  firebrats  fleas  silverfish and spiders. This product should be used as a spot treatment indoors along and behind baseboards  around and beneath appliances and beneath and behind sinks. Other areas where insects are often found include in cracks and crevices; around pipes and plumbing; around waste containers; in dark corners of rooms  cabinets and closets. To use this product  shake well before using. Spray insects to kill on contact. For long-lasting control  treat areas where insects are commonly found. Spray surfaces until wet  moving the spray rapidly to prevent excessive wetting and waste of product. When treating for ants  spray around door and window frames. Also treat other entry points and tr

In [37]:
len(docs)

5

In [176]:
def stemmed_inverted_index_from_data(
    doc_database # list[str,]
):
    terms_dictionary = {}
    for docID, doc in enumerate(doc_database): # int, str; list[(int, str), ]


        sentence_doc = word_tokenize(doc) # list[str,];
        # stems all words in doc
        sentence_str_1 = [p_stemmer.stem(word_tk) for word_tk in sentence_doc]  # list[str,]
        sentence_str_1 = " ".join(sentence_str_1) # str
        # subs all symbols/punctuations (including space) with a space (' '); excludes digits
        sentence_str_1 = re.sub(r'\W+', ' ',sentence_str_1)
        # subs all digits with a space (' '); a sequence of digits (without space) is given one (' ')
        sentence_str_1 = re.sub(r'\d+', ' ',sentence_str_1)
        # subs all symbols AND digits (including spaces) with a space (' '); includes digits
        sentence_str_1 = re.sub(r'[^a-zA-Z]', ' ',sentence_str_1)
        # compiles a pattern and subs out the pattern in sentence_str_1 with ' '
        sentence_str_1 = re.compile('(\s*)pron(\s*)').sub(' ', sentence_str_1)

        sentence_doc_1 = word_tokenize(sentence_str_1) # list[str,]
        # strip removes whitespace at beginning and end of string (not in between)
        sentence_str_2 = [str(word_tk_1).strip() for word_tk_1 in sentence_doc_1] # list[str,]
        # strings with >=2 chars only
        sentence_str_2  = [word_str for word_str in sentence_str_2 if len(word_str) >= 2] # list[str,]
        # strings which are not symbols/punctuations
        sentence_str_2 = [word_str for word_str in sentence_str_2  if word_str not in punctuation]
        # strings which are not stop-words; check with: Spacy_Eng.vocab['word'].is_stop
        sentence_str_2 = [word_str for word_str in sentence_str_2 if word_str not in stop_words]

        for term in sentence_str_2: # 'str'
            if term in terms_dictionary:
                terms_dictionary[term][0] += 1 # int; doc freq
                terms_dictionary[term][1].append(docID) # list[int,]
            else:
                terms_dictionary[term] =[[],[]] # list[[int], [int, ]]; list[[doc freq], [docIds,]]
                terms_dictionary[term][0] = 1 # int; doc freq
                terms_dictionary[term][1].append(docID)
    return terms_dictionary

In [162]:
doc1 = 'this is an apple, it is kind of fresh but also small, $2.99'
doc2 = 'never seen an orange like this one'
doc3 = 'these grapes are kind of small'

doc_database = [doc1, doc2, doc3]

In [182]:
inverted_index_dictionary = stemmed_inverted_index_from_data(doc_database)

In [228]:
def query_processing(
    query # str
):
    posting_list = [] # list[]
    query_terms = query.split() #list[str, ]
    each_term = 0 # int

    for term in query_terms: # str       

        edit_dist_list = [[key, nltk.edit_distance(term, key)]  for key in inverted_index_dictionary.keys()]
        edit_dist_list_1 = list(zip(*edit_dist_list)) # list[(str,)(int)]
        internal_query = edit_dist_list_1[0][edit_dist_list_1[1].index(min(edit_dist_list_1[1]))] # str

        posting_list.append(inverted_index_dictionary[internal_query][1]) # list[[int, ], ]
        each_term +=1
    for current_term in posting_list:
        for docIDof_cur_term in current_term:
            print(doc_database[docIDof_cur_term])
    return posting_list

In [227]:
query = 'appel seem' # 'str' typo; on both terms
all_postings = query_processing(query)

this is an apple, it is kind of fresh but also small, $2.99
never seen an orange like this one


In [ ]:
def or_postings(posting1, posting2):
  p1 = 0
  p2 = 0
  result = list()
  while p1 < len(posting1) and p2 < len(posting2):
    if posting1[p1] == posting2[p2]:
      result.append(posting1[p1])
      p1 += 1
      p2 += 2
    elif posting1[p1] > posting2[p2]:
      result.append(posting2[p2])
      p2 += 1
    else:
      result.append(posting1[p1])
      p1 += 1
  while p1 < len(posting1):
    result.append(posting1[p1])
    p1 += 1
  while p2 < len(posting2):
    result.append(posting2[p2])
    p2 += 1
    
  return result
